In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 21.5 MB/s eta 0:00:00


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd

In [ ]:
# Scraping all the links from the site-index and storing it as a JSON file
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def scrape_utdallas_site_links(url="https://www.utdallas.edu/siteindex/", output_file="scraped_utd_links.json"):
    """Scrapes all site links from the specified UTDallas site index page."""
    try:
        chrome_options = Options()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        driver = webdriver.Chrome(options=chrome_options)
        driver.get(url)

        # Scroll to the bottom of the page
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # Wait for content to load
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Wait for all links to be present.
        WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'a')))

        # Extract links using JavaScript
        links = driver.execute_script("""
            let linkElements = document.querySelectorAll('a');
            let links = [];
            for (let link of linkElements) {
                links.push({
                    text: link.textContent,
                    href: link.href
                });
            }
            return links;
        """)

        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(links, f, indent=4, ensure_ascii=False)
        print(f"Successfully scraped {len(links)} links and saved to {output_file}")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        if 'driver' in locals():
            driver.quit()

if __name__ == "__main__":
    scrape_utdallas_site_links()

Successfully scraped 1181 links and saved to scraped_utd_links.json


In [ ]:
# Data scraping - UTD
import os
import requests
from bs4 import BeautifulSoup
import time
import random

def scrape_and_store(input_json="scraped_utd_links.json", output_folder="scraped_sites_ind", combined_output="combined_scraped_data_utd.json"):
    """
    Reads links from a JSON, scrapes data from each link, stores as individual JSONs, and compiles into a single JSON.
    """
    try:
        with open(input_json, "r", encoding="utf-8") as f:
            links_data = json.load(f)
        links = [item["href"] for item in links_data]
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        scraped_data = {}
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        for i, link in enumerate(links):
            try:
                time.sleep(random.uniform(1, 3)) #delay to prevent rate limiting.
                response = requests.get(link, headers=headers)
                response.raise_for_status()
                soup = BeautifulSoup(response.content, "html.parser")
                text_content = soup.get_text(separator="\n", strip=True)
                filename = os.path.join(output_folder, f"site_{i}.json")
                with open(filename, "w", encoding="utf-8") as f:
                    json.dump({"url": link, "content": text_content}, f, indent=4, ensure_ascii=False)
                scraped_data[link] = {"url": link, "content": text_content, "children": {}}
            except requests.exceptions.RequestException as e:
                print(f"Error accessing {link}: {e}")
                scraped_data[link] = {"url": link, "content": f"Error accessing {link}: {e}", "children": {}}
        with open(combined_output, "w", encoding="utf-8") as f:
            json.dump(scraped_data, f, indent=4, ensure_ascii=False)
        print(f"Scraped and stored data in {output_folder} and {combined_output}")
    except FileNotFoundError:
        print(f"Error: {input_json} not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

scrape_and_store()

Error accessing https://outlook.office365.com/owa/?realm=utdallas.edu: ('Connection aborted.', HTTPException('got more than 100 headers'))
Error accessing : Invalid URL '': No scheme supplied. Perhaps you meant https://?
Error accessing https://urec.utdallas.edu//facilities/index.html#ac: 404 Client Error: Not Found for url: https://urec.utdallas.edu//facilities/index.html#ac
Error accessing : Invalid URL '': No scheme supplied. Perhaps you meant https://?
Error accessing : Invalid URL '': No scheme supplied. Perhaps you meant https://?


Error accessing https://www.collin.edu/chec/: 404 Client Error: Not Found for url: https://www.collin.edu/chec/
Error accessing : Invalid URL '': No scheme supplied. Perhaps you meant https://?
Error accessing : Invalid URL '': No scheme supplied. Perhaps you meant https://?
Error accessing https://outlook.office365.com/owa/?realm=utdallas.edu: ('Connection aborted.', HTTPException('got more than 100 headers'))
Error accessing : Invalid URL '': No scheme supplied. Perhaps you meant https://?
Error accessing : Invalid URL '': No scheme supplied. Perhaps you meant https://?
Error accessing : Invalid URL '': No scheme supplied. Perhaps you meant https://?
Error accessing : Invalid URL '': No scheme supplied. Perhaps you meant https://?
Error accessing https://news.utdallas.edu/campus-community/inclement-weather-policy-2022/: HTTPSConnectionPool(host='news-2024.utdallas.edu', port=443): Max retries exceeded with url: /campus-community/inclement-weather-closing-policies-2023/ (Caused by Nam

In [ ]:
# Code to determine the number of unique utd links

import os
import json

def count_json_files(folder_path="scraped_sites_ind"):
    """
    Counts the number of JSON files in the specified folder.
    """
    try:
        if not os.path.exists(folder_path):
            print(f"Folder '{folder_path}' does not exist.")
            return 0

        file_count = 0
        for filename in os.listdir(folder_path):
            if filename.endswith(".json") and os.path.isfile(os.path.join(folder_path, filename)):
                file_count += 1

        print(f"Number of JSON files in '{folder_path}': {file_count}")
        return file_count

    except Exception as e:
        print(f"An error occurred: {e}")
        return 0

# Example usage in Google Colab:
count_json_files()

Number of JSON files in 'scraped_sites_ind': 1144


1144

In [ ]:
# Code to save all the necessary links as a json file.

import pandas as pd
import json

# Define the input Excel file name and output JSON file name
excel_file = "Links_Necessary.xlsx"
json_file = "Links_Necessary.json"

try:
    # Read the Excel file into a pandas DataFrame
    df = pd.read_excel(excel_file)

    # Assuming the links are in the first column (index 0).
    # Adjust the column index if your links are in a different column.
    links_column = df.iloc[:, 0].astype(str)

    # Remove serial numbers and extract only the links
    extracted_links = []
    for item in links_column:
        # Split the string by the first space to separate the serial number (if present)
        parts = item.split(' ', 1)
        if len(parts) > 1:
            link = parts[1].strip()
            extracted_links.append(link)
        else:
            # If no space is found, assume the whole item is the link
            extracted_links.append(item.strip())

    # Remove duplicate links by converting to a set and back to a list
    unique_links = sorted(list(set(extracted_links)))

    # Get the count of unique links
    link_count = len(unique_links)

    # Save the unique links to a JSON file
    data = {"links": unique_links}
    with open(json_file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

    print(f"Successfully extracted {link_count} unique links from '{excel_file}' and saved them to '{json_file}'.")

except FileNotFoundError:
    print(f"Error: The file '{excel_file}' was not found. Please make sure the file is in the correct directory.")
except Exception as e:
    print(f"An error occurred: {e}")
    print(f"Error details: {e}")

Successfully extracted 223 unique links from 'Links_Necessary.xlsx' and saved them to 'Links_Necessary.json'.


In [ ]:
# Data scraping - Links_Necessary

import os
import requests
from bs4 import BeautifulSoup
import time
import random
import json
from urllib.parse import urlparse, urljoin

def scrape_and_store_combined(input_json="Links_Necessary.json", combined_output="combined_scraped_data_links_necessary.json"):
    """
    Reads links from a JSON file with the structure {"links": [...]},
    scrapes data from each link, and identifies/scrapes sub-links based on
    whether they are within the same domain and path structure.
    All scraped data is compiled into a single nested JSON file.
    """
    try:
        with open(input_json, "r", encoding="utf-8") as f:
            data = json.load(f)
            initial_links = data.get("links", [])
            if not initial_links:
                print(f"Warning: No links found in '{input_json}'.")
                return
        scraped_data = {}
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }

        def should_be_sublink(main_url, potential_sub_url):
            """
            Determines if a potential URL should be considered a sub-link
            of the main URL based on domain and path.
            """
            main_parsed = urlparse(main_url)
            sub_parsed = urlparse(potential_sub_url)

            if main_parsed.netloc == sub_parsed.netloc and potential_sub_url.startswith(main_url):
                return True
            return False

        def scrape_page(url, parent_data, scraped):
            """
            Scrapes the content of the given URL and recursively scrapes its sub-links.
            """
            if url in scraped:
                return  # Avoid re-scraping

            try:
                time.sleep(random.uniform(1, 3))  # Delay to prevent rate limiting.
                response = requests.get(url, headers=headers)
                response.raise_for_status()
                soup = BeautifulSoup(response.content, "html.parser")
                text_content = soup.get_text(separator="\n", strip=True)
                parent_data["content"] = text_content
                parent_data["children"] = {}
                scraped.add(url)

                # Find all links on the page
                all_links = [urljoin(url, a.get("href")) for a in soup.find_all("a", href=True)]
                unique_links_on_page = list(set(all_links))

                sub_link_counter = 0
                for potential_sub_link in unique_links_on_page:
                    if should_be_sublink(url, potential_sub_link) and potential_sub_link != url:
                        child_data = {"url": potential_sub_link, "content": "", "children": {}}
                        parent_data["children"][f"sub_link_{sub_link_counter}"] = child_data
                        scrape_page(potential_sub_link, child_data, scraped)
                        sub_link_counter += 1

            except requests.exceptions.RequestException as e:
                parent_data["content"] = f"Error accessing {url}: {e}"
                parent_data["children"] = {}
                print(f"Error accessing {url}: {e}")
            except Exception as e:
                parent_data["content"] = f"Error processing {url}: {e}"
                parent_data["children"] = {}
                print(f"Error processing {url}: {e}")

        scraped_urls = set()
        for i, main_link in enumerate(initial_links):
            if isinstance(main_link, str) and main_link.startswith("http"):
                main_link_data = {"url": main_link, "content": "", "children": {}}
                scraped_data[f"main_link_{i}"] = main_link_data
                scrape_page(main_link, main_link_data, scraped_urls)
            else:
                print(f"Warning: Skipping invalid link in input: {main_link}")

        with open(combined_output, "w", encoding="utf-8") as f:
            json.dump(scraped_data, f, indent=4, ensure_ascii=False)
        print(f"Scraped and stored data in {combined_output}")

    except FileNotFoundError:
        print(f"Error: {input_json} not found.")
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from '{input_json}'. Please ensure the file is valid JSON.")
    except Exception as e:
        print(f"An error occurred: {e}")

scrape_and_store_combined()